Idea is to read in the annotation file of emapper.py and pull out the annotations.
This was run like:
python /home/benjamin/anaconda3/envs/funannotate/eggnog-mapper-0.99.2/emapper.py -i ../DK_0911_v01_h_ctg.proteiniprs.fa -d euk --output DK_0911_v01_h_ctg --cpu 12


In [1]:
import pandas as pd
import os
import re
from Bio import SeqIO
import pysam
from Bio.SeqRecord import SeqRecord
from pybedtools import BedTool
import numpy as np
import pybedtools
import time
import matplotlib.pyplot as plt
import sys
import subprocess
import shutil

In [3]:
p_genome = 'DK_0911_v01_h_ctg'
p_genome_filtered = 'DK_0911_v04LT_h_ctg'
BASE_FOLDER = '/home/benjamin/genome_assembly/Warrior/annotation/%s/eggnog/' % p_genome
EGGNOG_BLAST_FILE = '%s.emapper.annotations' % (p_genome)
#EGGNOG_DIAMON_FILE = '%s_diamond.emapper.annotations' % (p_genome)
BASE_AA_PATH = '/home/benjamin/genome_assembly/Warrior/DK0911_v04'
BASE_A_PATH = '/home/benjamin/genome_assembly/Warrior/genome_v04'
OUT_PATH = os.path.join(BASE_AA_PATH, 'protein_annotation',p_genome_filtered, 'eggnog-mapper', 'parsed')
EGGNOG_PATH = OUT_PATH
INTERPRO_PATH = os.path.join(BASE_AA_PATH, 'protein_annotation',p_genome_filtered, 'interpro', 'parsed')
FUN_PATH = os.path.join(BASE_AA_PATH, 'protein_annotation',p_genome_filtered, 'funannotate', 'parsed')
COMB_OUT_PATH = os.path.join(BASE_AA_PATH, 'protein_annotation',p_genome_filtered, 'combined')
if not os.path.exists(OUT_PATH):
    os.makedirs(OUT_PATH)
if not os.path.exists(OUT_PATH):
    os.makedirs(OUT_PATH)
if not os.path.exists(COMB_OUT_PATH):
    os.makedirs(COMB_OUT_PATH)
if not os.path.exists(FUN_PATH):
    print('Please run funnanotate notebook before running the last steps of this notebook.')
if not os.path.exists(INTERPRO_PATH):
    print('Please run the interpro parser notebook before running the last stpes of this notebook.')

In [4]:
#pull out all proteins that are in the final assembly
p_protein_list = []
protein_fa_file = [x for x in os.listdir(BASE_A_PATH) if p_genome_filtered in x and x.endswith('.protein.fa')][0]
for protein in SeqIO.parse(os.path.join(BASE_A_PATH, protein_fa_file) , 'fasta'):
    p_protein_list.append(protein.id)

In [5]:
eggnog_blast_header = 'query_name seed_eggNOG_ortholog seed_ortholog_evalue seed_ortholog_score predicted_gene_name \
GO_terms KEGG_pathways Annotation_tax_scope OGs bestOG|evalue|score COG cat eggNOG annot'.split(' ')

In [6]:
eggnog_blast_df = pd.read_csv(os.path.join(BASE_FOLDER, EGGNOG_BLAST_FILE), sep ='\t', header=None, names=eggnog_blast_header, skiprows=3)

In [7]:
eggnog_blast_df.shape

(7252, 14)

In [9]:
eggnog_blast_df.fillna(0, inplace =True)

In [10]:
#here filter out all the proteins that are TE related and not in genome version v04 
eggnog_blast_df = eggnog_blast_df[eggnog_blast_df.query_name.isin(p_protein_list)].reset_index(drop=True)

In [11]:
eggnog_blast_df.shape

(5202, 14)

In [12]:
#pick all annotation columns. One at a time and write them out as tab file or annotations files. The later can be used
#to annotate gff files using gag.py
DBs = [x for x in eggnog_blast_df.columns.tolist()[4:] if x not in ['GO_terms','KEGG_pathways', 'OGs','bestOG|evalue|score' ] ]
eggnog_blast_df['note'] = 'note'
for db in DBs:
    if len(eggnog_blast_df[eggnog_blast_df[db] !=0]) > 0:
        eggnog_blast_df[eggnog_blast_df[db] !=0].loc[:,['query_name',db]]\
        .to_csv(os.path.join(OUT_PATH, db+'_terms.tab'), sep='\t', header =None, index = None)
        eggnog_blast_df[eggnog_blast_df[db] !=0].loc[:,['query_name', 'note', db]]\
        .to_csv(os.path.join(OUT_PATH, 'annotations.' +db+'.txt'), sep='\t', header =None, index = None)
    
    

In [13]:
#now look after the remaining columns which have potentially multiple entries as values eg. GOterm, GOterm, GOterm
#those values need to be split into a list and this made into a new dataframe before writting it out again.
eggnog_blast_df = pd.read_csv(os.path.join(BASE_FOLDER, EGGNOG_BLAST_FILE), sep ='\t', header=None, names=eggnog_blast_header, skiprows=3)
eggnog_blast_df = eggnog_blast_df[eggnog_blast_df.query_name.isin(p_protein_list)].reset_index(drop=True)
#['GO_terms','KEGG_pathways', 'OGs']
for db in ['GO_terms','KEGG_pathways', 'OGs']:
    eggnog_blast_df[db] = eggnog_blast_df[db].str.split(',')
    all_term_list = []
    all_query_list = []
    eggnog_blast_df[db].fillna(0, inplace=True)
    for key in eggnog_blast_df[eggnog_blast_df[db] != 0 ].index:
        for term in eggnog_blast_df.loc[key,db]:
            all_term_list.append(term)
            all_query_list.append(eggnog_blast_df.loc[key,'query_name'])
    tmp_df = pd.DataFrame([all_query_list, all_term_list]).T
    tmp_df.rename(columns={0:'query_name', 1:'DB_ID'}, inplace=True)
    tmp_df.to_csv(os.path.join(OUT_PATH, db+'_terms.tab'), sep='\t', header =None, index = None)
    tmp_df['note'] = 'note'
    tmp_df.loc[:, ['query_name', 'note', 'DB_ID']].to_csv(os.path.join(OUT_PATH, 'annotations.' +db+'.txt'), sep='\t', header =None, index = None)

In [14]:
#combine KEGG annotations, and GO terms
egg_GO_file = [os.path.join(OUT_PATH, x) for x in os.listdir(OUT_PATH) if x == 'annotations.GO_terms.txt'][0] 
egg_KEGG_file = [os.path.join(OUT_PATH, x) for x in os.listdir(OUT_PATH) if x == 'annotations.KEGG_pathways.txt'][0]
interpro_GO_file = [os.path.join(INTERPRO_PATH, x)\
                      for x in os.listdir(INTERPRO_PATH) if x == 'annotations.GO.txt'][0]
interpro_KEGG_file = [os.path.join(INTERPRO_PATH, x)\
                      for x in os.listdir(INTERPRO_PATH) if x == 'annotations.Pathway.txt'][0]

In [15]:
GO_df = pd.concat([pd.read_csv(egg_GO_file, header=None, sep='\t'), pd.read_csv(interpro_GO_file, header=None, sep='\t')])

In [16]:
GO_df.drop_duplicates().to_csv(os.path.join(OUT_PATH, 'annotations.GO_combined.txt'), header=None, index =None, sep ='\t')

In [17]:
interpro_pathway_df = pd.read_csv(interpro_KEGG_file, header = None, sep='\t', names=['query_name','note', 'DB'])

In [18]:
interpro_pathway_df.head()

,query_name,note,DB
0,DK0911_19966,note,Reactome: R-HSA-196757
1,DK0911_22885,note,Reactome: R-HSA-191859
2,DK0911_24876,note,MetaCyc: PWY-5514
3,DK0911_24876,note,KEGG: 00520+5.4.2.10
4,DK0911_24876,note,MetaCyc: PWY-6906


In [19]:
interpro_KEGG_df = interpro_pathway_df[interpro_pathway_df.DB.str.contains('KEGG')]

In [20]:
interpro_KEGG_df['new_KEGG'] = 'map'+interpro_KEGG_df.DB.str.extract(r'KEGG: ([0-9]+)+')

/home/benjamin/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  if __name__ == '__main__':
/home/benjamin/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [21]:
KEGG_df = pd.concat([interpro_KEGG_df.loc[:,['query_name', 'note', 'new_KEGG']],\
                     pd.read_csv(egg_KEGG_file, header=None, sep='\t', names=['query_name','note', 'new_KEGG'])])

In [22]:
KEGG_df.drop_duplicates().to_csv(os.path.join(OUT_PATH, 'annotations.KEGG_combined.txt'), header=None, index =None, sep ='\t')

In [23]:
interpro_pathway_df[~interpro_pathway_df.DB.str.contains('KEGG')].to_csv(os.path.join(INTERPRO_PATH,\
                                                                         'annotations.Pathway_no_KEGG.txt'), header=None, index=None, sep='\t')

In [24]:
#filter out other annotations files including busco, dbCAN, merops, swissprot
funannotate_files = [os.path.join(FUN_PATH, x) for x in os.listdir(FUN_PATH) \
                    if x.startswith('annotations') and (x.endswith('busco.txt') or x.endswith('dbCAN.txt') or x.endswith('merops.txt') or x.endswith('swissprot.txt'))]
#now get all eggnog files to write out the annotations as 'gene ID;ID'
eggnog_files = [os.path.join(EGGNOG_PATH, x) for x in os.listdir(EGGNOG_PATH) if x.startswith('annotations')\
                and ('GO_terms' not in x and 'KEGG_pathways' not in x)]
interpro_files = [os.path.join(INTERPRO_PATH, x) for x in os.listdir(INTERPRO_PATH) if x.startswith('annotations')\
                 and ('.GO.' not in x and '.Pathway.' not in x)]

In [25]:
#do the writing out as 'gene \t ID;ID'
for file in (funannotate_files + eggnog_files + interpro_files):
    tmp_df = pd.read_csv(file, header = None, sep ='\t', names=['query_name', 'note', 'annotation'])
    tmp_dict = {}
    #now do a quick check if the annotations  contains some DATABASE:ID description
    #if that is the case remove the database name.
    if any(':' in x for x in tmp_df['annotation']):
        for name, group in tmp_df.groupby('query_name'):
            tmp_list = group['annotation'].tolist()
            for index, value in enumerate(tmp_list):
                if ':' in value:
                    tmp_list[index] = value[value.index(':')+1:].lstrip()
            tmp_dict[name] = ';'.join(tmp_list)
    else:
        for name, group in tmp_df.groupby('query_name'):
            tmp_dict[name] = ';'.join(group['annotation'].tolist())
    fn = file.split('/')[-1]
    fn = fn.replace('annotations', p_genome_filtered)
    fn = fn.replace('.txt', '.tablist')
    fn = os.path.join(COMB_OUT_PATH, fn)
    tmp_fn = open(fn, 'w')
    for key, value in tmp_dict.items():
        print('%s\t%s' %(key, value) ,file=tmp_fn)
    tmp_fn.close()